## Diebold Mariano Test
1. Data Preparation: Load Forecasts and Actual Values for h = 1, 4, 12
2. Check if errors are covariance stationary
3. If stationary perform DB test against the Benchmark (Naive Mean Model)

In [1]:
# NB config
%load_ext autoreload
%autoreload 2

# Load Libraries
import os
import importlib
import warnings

os.chdir("../../")
print(os.getcwd())
import numpy as np
import pandas as pd
from pathlib import Path
from darts import TimeSeries
from darts.metrics import (
    rmse,
)
from arch.unitroot import ADF, DFGLS, KPSS
from pprint import pprint

dm_test = importlib.import_module(
    "forecasting.time_series_tests.Diebold-Mariano-Test.dm_test"
)

warnings.filterwarnings("ignore")

/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index


/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/.venv/crypto-uncertainty-index-SX872G0E-py3.8/lib/python3.8/site-packages/darts/timeseries.py:39: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.DatetimeIndex, pd.RangeIndex, pd.Int64Index)
/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/.venv/crypto-uncertainty-index-SX872G0E-py3.8/lib/python3.8/site-packages/darts/timeseries.py:512: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  times: Union[pd.DatetimeIndex, pd.Int64Index],
/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/.venv/crypto-uncertainty-index-SX872G0E-py3.8/lib/python3.8/site-packages/darts/timeseries.py:738: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a fu

In [2]:
# Data Dir
data_dir = Path("forecasting/data/modelling")

# BTC-USD data
btc_usd_fp = data_dir / "btc_usd_weekly.csv"
btc_usd_df = pd.read_csv(btc_usd_fp)

# h = 1 (Weekly Price Returns)
btc_usd_df["Price Returns (h=1)"] = np.log1p(btc_usd_df[["Price"]].pct_change(1))

# h = 4 (4 Week Price Returns)
btc_usd_df["Price Returns (h=4)"] = np.log1p(btc_usd_df[["Price"]].pct_change(4))

# h = 12 (12 Week Price Returns)
btc_usd_df["Price Returns (h=12)"] = np.log1p(btc_usd_df[["Price"]].pct_change(12))

# Create TimeSeries
# h = 1 (Weekly Price Returns)
btc_usd1_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=1)"]].dropna(), time_col="Date"
)

# h = 4 (4 Week Price Returns)
btc_usd4_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=4)"]].dropna(), time_col="Date"
)

# h = 12 (12 Week Price Returns)
btc_usd12_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=12)"]].dropna(), time_col="Date"
)

### Load Forecasts

In [3]:
rf_forecasts_dir = Path("forecasting/data/forecasts/random_forest")

# Model A
rfA1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_A" / "rf_model_A_h1.csv", time_col="time"
)
rfA4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_A" / "rf_model_A_h4.csv", time_col="time"
)
rfA12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_A" / "rf_model_A_h12.csv", time_col="time"
)

# Model B
rfB1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_B" / "rf_model_B_h1.csv", time_col="time"
)
rfB4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_B" / "rf_model_B_h4.csv", time_col="time"
)
rfB12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_B" / "rf_model_B_h12.csv", time_col="time"
)

# Model C
rfC1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_C" / "rf_model_C_h1.csv", time_col="time"
)
rfC4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_C" / "rf_model_C_h4.csv", time_col="time"
)
rfC12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_C" / "rf_model_C_h12.csv", time_col="time"
)

# Model D
rfD1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_D" / "rf_model_D_h1.csv", time_col="time"
)
rfD4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_D" / "rf_model_D_h4.csv", time_col="time"
)
rfD12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_D" / "rf_model_D_h12.csv", time_col="time"
)

# Model E
rfE1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_E" / "rf_model_E_h1.csv", time_col="time"
)
rfE4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_E" / "rf_model_E_h4.csv", time_col="time"
)
rfE12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_E" / "rf_model_E_h12.csv", time_col="time"
)

# Model F
rfF1_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_F" / "rf_model_F_h1.csv", time_col="time"
)
rfF4_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_F" / "rf_model_F_h4.csv", time_col="time"
)
rfF12_forecast = TimeSeries.from_csv(
    rf_forecasts_dir / "model_F" / "rf_model_F_h12.csv", time_col="time"
)

In [4]:
# Benchmark
bench_forecast_dir = Path("forecasting/data/forecasts/mean_model")

mean1_forecast = TimeSeries.from_csv(
    bench_forecast_dir / "mean_model_h1.csv", time_col="time"
)
mean4_forecast = TimeSeries.from_csv(
    bench_forecast_dir / "mean_model_h4.csv", time_col="time"
)
mean12_forecast = TimeSeries.from_csv(
    bench_forecast_dir / "mean_model_h12.csv", time_col="time"
)

In [17]:
def mse_custom(y_actual: TimeSeries, y_pred: TimeSeries):
    y_actual_red = y_actual.slice_intersect(y_pred).all_values().squeeze(-1)
    y_pred_red = y_pred.all_values().squeeze(-1)
    mse = (y_actual_red - y_pred_red) ** 2
    return mse

### Check Loss Differential Covariance Stationarity

#### 1. h1 models

In [19]:
# Compute H1 Losses

bench1_loss = mse_custom(btc_usd1_ts, mean1_forecast)
rfA1_loss = mse_custom(btc_usd1_ts, rfA1_forecast)
rfB1_loss = mse_custom(btc_usd1_ts, rfB1_forecast)
rfC1_loss = mse_custom(btc_usd1_ts, rfC1_forecast)
rfD1_loss = mse_custom(btc_usd1_ts, rfD1_forecast)
rfE1_loss = mse_custom(btc_usd1_ts, rfE1_forecast)
rfF1_loss = mse_custom(btc_usd1_ts, rfF1_forecast)

h1_names = ["benchmark", "RF-A", "RF-B", "RF-C", "RF-D", "RF-E", "RF-F"]
h1_losses = [
    rfA1_loss,
    rfB1_loss,
    rfC1_loss,
    rfD1_loss,
    rfE1_loss,
    rfF1_loss,
    bench1_loss,
]

In [20]:
# Check Stationarity
h1_cov_stat_res = {}

for name, i in zip(h1_names, h1_losses):
    computed = DFGLS(i)
    summary = {
        "stat": computed.stat,
        "p_val": computed.pvalue,
        "num_lags": computed.lags,
        "is_stationary": computed.pvalue < 0.05,
    }
    h1_cov_stat_res[name] = summary

pprint(h1_cov_stat_res)

{'RF-A': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 5.172068710681032e-17,
          'stat': -9.999853411787617},
 'RF-B': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 1.425797375440484e-17,
          'stat': -10.286097678437486},
 'RF-C': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 4.331680096277558e-17,
          'stat': -10.038894573865408},
 'RF-D': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 9.269312766980521e-17,
          'stat': -9.872121420379186},
 'RF-E': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 3.685236559596351e-17,
          'stat': -10.074572992938437},
 'RF-F': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 1.6849156796746067e-16,
          'stat': -9.742382828709273},
 'benchmark': {'is_stationary': True,
               'num_lags': 0,
               'p_val': 6.344495682848364e-17,
               'stat': -9.954997782358976}}


#### 2. h4 models

In [21]:
# Compute H4 Losses

bench4_loss = mse_custom(btc_usd4_ts, mean4_forecast)
rfA4_loss = mse_custom(btc_usd4_ts, rfA4_forecast)
rfB4_loss = mse_custom(btc_usd4_ts, rfB4_forecast)
rfC4_loss = mse_custom(btc_usd4_ts, rfC4_forecast)
rfD4_loss = mse_custom(btc_usd4_ts, rfD4_forecast)
rfE4_loss = mse_custom(btc_usd4_ts, rfE4_forecast)
rfF4_loss = mse_custom(btc_usd4_ts, rfF4_forecast)

h4_names = ["benchmark", "RF-A", "RF-B", "RF-C", "RF-D", "RF-E", "RF-F"]
h4_losses = [
    rfA4_loss,
    rfB4_loss,
    rfC4_loss,
    rfD4_loss,
    rfE4_loss,
    rfF4_loss,
    bench4_loss,
]

In [22]:
# Check Stationarity
h4_cov_stat_res = {}

for name, i in zip(h4_names, h4_losses):
    computed = DFGLS(i)
    summary = {
        "stat": computed.stat,
        "p_val": computed.pvalue,
        "num_lags": computed.lags,
        "is_stationary": computed.pvalue < 0.05,
    }
    h4_cov_stat_res[name] = summary

pprint(h4_cov_stat_res)

{'RF-A': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 3.29725323614587e-13,
          'stat': -8.162629820379635},
 'RF-B': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 2.8013589633756594e-12,
          'stat': -7.72861401073712},
 'RF-C': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 1.495161828548182e-12,
          'stat': -7.8557989488988875},
 'RF-D': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 1.6134730938826566e-12,
          'stat': -7.840367486119828},
 'RF-E': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 5.272817801317614e-12,
          'stat': -7.6005584212679},
 'RF-F': {'is_stationary': True,
          'num_lags': 1,
          'p_val': 2.495638125548336e-10,
          'stat': -6.817152485821982},
 'benchmark': {'is_stationary': True,
               'num_lags': 0,
               'p_val': 1.2170999919668754e-12,
               'stat': -7.897502579220049}}


#### 3. h12 models

In [23]:
# Compute H12 Losses

bench12_loss = mse_custom(btc_usd12_ts, mean12_forecast)
rfA12_loss = mse_custom(btc_usd12_ts, rfA12_forecast)
rfB12_loss = mse_custom(btc_usd12_ts, rfB12_forecast)
rfC12_loss = mse_custom(btc_usd12_ts, rfC12_forecast)
rfD12_loss = mse_custom(btc_usd12_ts, rfD12_forecast)
rfE12_loss = mse_custom(btc_usd12_ts, rfE12_forecast)
rfF12_loss = mse_custom(btc_usd12_ts, rfF12_forecast)

h12_names = ["benchmark", "RF-A", "RF-B", "RF-C", "RF-D", "RF-E", "RF-F"]
h12_losses = [
    rfA12_loss,
    rfB12_loss,
    rfC12_loss,
    rfD12_loss,
    rfE12_loss,
    rfF12_loss,
    bench12_loss,
]

In [24]:
# Check Stationarity
h12_cov_stat_res = {}

for name, i in zip(h12_names, h12_losses):
    computed = DFGLS(i)
    summary = {
        "stat": computed.stat,
        "p_val": computed.pvalue,
        "num_lags": computed.lags,
        "is_stationary": computed.pvalue < 0.05,
    }
    h12_cov_stat_res[name] = summary

pprint(h12_cov_stat_res)

{'RF-A': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 4.87174246392154e-13,
          'stat': -8.083286035911724},
 'RF-B': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 9.33248022506735e-13,
          'stat': -7.9513470667898165},
 'RF-C': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 9.058506357552613e-13,
          'stat': -7.957390430772302},
 'RF-D': {'is_stationary': True,
          'num_lags': 1,
          'p_val': 1.5694477933406433e-08,
          'stat': -5.958055470837175},
 'RF-E': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 9.641843795249509e-11,
          'stat': -7.011122546974434},
 'RF-F': {'is_stationary': True,
          'num_lags': 0,
          'p_val': 0.010407773083322787,
          'stat': -2.56698739979719},
 'benchmark': {'is_stationary': True,
               'num_lags': 0,
               'p_val': 1.6373088473036544e-13,
               'stat': -8.305183716013877}}


### Diebold Mariano Test for Equivalent Forecasts

#### 1. H1 models

In [ ]:
# Concat Timeseries data to ensure that they match in terms of time frame

#### 2. H4 models

#### 3. H12 models